# Deploy Watson ML (PMML)

Deploys a PMML model to IBM Watson Machine Learning (WML)

In [1]:
#%pip install ibm-watson-machine-learning
# PLEASE RESTART YOUR KERNAL AFTER THIS LINE HAS BEEN EXECUTED

In [1]:
import wget
from ibm_watson_machine_learning import APIClient
import os
import sys

In [2]:
# IBM Cloud API Key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key',' <insert your api key>')

# Machine Learning Model Deployment Space https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space','<insert your space id>')

# IBM Cloud Region (e.g. us-south)
location = os.environ.get('location','<insert your location>')

# temporary directory for data
data_dir = os.environ.get('data_dir',
                          '../../data/')

In [4]:
parameters = list(
  map(
      lambda s: re.sub('$', '"', s),
      map(
          lambda s: s.replace('=', '="'),
          filter(
              lambda s: s.find('=') > -1 and bool(re.match('[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
              sys.argv
          )
      )
  )
)

for parameter in parameters:
    logging.warning('Parameter: '+parameter) 
    exec(parameter)

In [3]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [12]:
#wml_credentials

In [7]:
client = APIClient(wml_credentials)

In [8]:
o = client.software_specifications.get_uid_by_name('spark-mllib_2.4')
software_spec_uid = o
client.set.default_space(space)

'SUCCESS'

In [3]:
#!pip install ibm-watson-machine-learning==1.0.45

In [10]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: 'test_pmml2',
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model=data_dir + 'model.xml',
    meta_props=model_meta_props,
)

model_uid = client.repository.get_model_uid(published_model)

In [5]:
import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = "<insert your api key>"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
#payload_scoring = {"input_data": [{"fields": [array_of_input_fields], "values": [array_of_values_to_be_scored, another_array_of_values_to_be_scored]}]}
payload_scoring = {"input_data": [{"fields": ["x", "y", "z", ], "values": [[1,2,3]]}]}


response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/3df3f301-cbc2-456d-bcfb-182739922630/predictions?version=2021-10-08&version=2021-10-08', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())

Scoring response
{'predictions': [{'fields': ['prediction', 'probability(Walk)', 'probability(Getup_bed)', 'probability(Drink_glass)', 'probability(Pour_water)', 'probability(Climb_stairs)', 'probability(Eat_meat)', 'probability(Brush_teeth)', 'probability(Standup_chair)', 'probability(Sitdown_chair)', 'probability(Comb_hair)', 'probability(Descend_stairs)', 'probability(Use_telephone)', 'probability(Liedown_bed)', 'probability(Eat_soup)'], 'values': [[0.0, 0.2066304733429463, 0.1027003496303976, 0.09607288600980918, 0.09341128039059202, 0.08988767566243888, 0.0701038367411341, 0.06675383158684296, 0.05710178483844282, 0.05594042524223481, 0.052495528411078754, 0.034243162220939566, 0.034097642928087285, 0.025596033031672127, 0.014965089963383691]]}]}
